# 노트북3: 구조 예측 결과 분석 및 결합력 평가 (PLIP, Pafnucy 등)

In [ ]:

!apt-get install -y openbabel
!pip install -q plip
!git clone https://github.com/oddt/pafnucy.git
%cd pafnucy
!pip install -q -r requirements.txt
%cd ..

from plip.structure.preparation import PDBComplex
import pandas as pd
import os

N = 5  # 펩타이드 후보 개수 맞게 수정

results = []

for i in range(N):
    pred_pdb = f"prediction_complex_{i}/complex_{i}_0.pdb"
    if not os.path.exists(pred_pdb):
        continue

    os.system(f"obabel {pred_pdb} -O receptor_{i}.pdbqt")
    os.system(f"cp receptor_{i}.pdbqt ligand_{i}.pdbqt")

    if not os.path.exists("vina_1.2.3_linux_x86_64/vina"):
        !wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64.zip
        !unzip -q vina_1.2.3_linux_x86_64.zip
        !chmod +x vina_1.2.3_linux_x86_64/vina

    vina_cmd = f"./vina_1.2.3_linux_x86_64/vina --receptor receptor_{i}.pdbqt --ligand ligand_{i}.pdbqt --center_x 0 --center_y 0 --center_z 0 --size_x 20 --size_y 20 --size_z 20 --out output_{i}.pdbqt --log log_{i}.txt"
    os.system(vina_cmd)

    vina_score = None
    with open(f"log_{i}.txt") as f:
        for line in f:
            if "REMARK VINA RESULT" in line:
                vina_score = float(line.strip().split()[3])
                break

    # PLIP 상호작용 분석
    structure = PDBComplex()
    structure.load_pdb(f"output_{i}.pdbqt")
    structure.analyze()
    interaction_count = 0
    for ligand in structure.ligands:
        inter = structure.interaction_sets[ligand]
        interaction_count += len(inter.hbonds) + len(inter.hydrophobic_contacts) + len(inter.saltbridge_ligands)

    # Pafnucy 평가
    os.system(f"obabel output_{i}.pdbqt -O complex_final_{i}.pdb")
    os.system(f"python pafnucy/predict.py --pdb complex_final_{i}.pdb --out affinity_{i}.csv")
    pafnucy_df = pd.read_csv(f"affinity_{i}.csv")
    pafnucy_affinity = float(pafnucy_df['predicted_affinity'].iloc[0])

    final_score = (-1 * vina_score) + (-1 * pafnucy_affinity) + (0.5 * interaction_count)
    results.append({
        "index": i,
        "vina_score": vina_score,
        "pafnucy": pafnucy_affinity,
        "interaction": interaction_count,
        "final_score": final_score
    })

df = pd.DataFrame(results)
df_sorted = df.sort_values("final_score", ascending=False)
df_sorted.to_csv("peptide_binding_rank.csv", index=False)
print("✅ 랭킹 결과 (상위 5개):")
display(df_sorted.head())
print("전체 결과 파일 저장 완료: peptide_binding_rank.csv")
